In [12]:
# Assume content_img and style_img are already loaded as 4D tensors [1, 3, H, W]
# with normalized values (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

vgg = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=True).features.eval()

content_layers = ['21']
style_layers = ['0', '5', '10', '19', '28']

def get_features(x, model, layers):
    features = {}
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[name] = x
    return features

def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(c, h * w)
    gram = torch.mm(features, features.t())
    return gram / (c * h * w)

def de_normalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1)
    return (tensor * std + mean).clamp(0,1)

def neural_style_transfer(content_img, style_img, steps=300, alpha=1e4, beta=1):
    target = content_img.clone().requires_grad_(True)

    style_features = get_features(style_img, vgg, style_layers)
    content_features = get_features(content_img, vgg, content_layers)
    style_grams = {l: gram_matrix(style_features[l]) for l in style_layers}

    optimizer = torch.optim.Adam([target], lr=0.003)

    for i in range(steps):
        target_features = get_features(target, vgg, style_layers + content_layers)

        c_loss = torch.mean((target_features['21'] - content_features['21']) ** 2)

        s_loss = 0
        for layer in style_layers:
            target_gram = gram_matrix(target_features[layer])
            s_loss += torch.mean((target_gram - style_grams[layer]) ** 2)

        total_loss = alpha * c_loss + beta * s_loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print(f"Step {i}, Loss: {total_loss.item()}")

    return de_normalize(target)

Using cache found in C:\Users\gunav/.cache\torch\hub\pytorch_vision_v0.10.0
